**This notebook allows you to run the trading game with your pricing model and see how well it fares!**

The general outline is the following:

You are a market maker trying to stay in the market with. What does this mean? The underlying of the option, the spot, constantly changes and with it the value of all the options change. Here we use a very simple model market where we have one spot and just a single option that is traded.

We have the following actors:

**UnderlyingMarket**: this spits out new spot values at regular intervals and also at the same time updates the quote of the single option that is trading.

**MarketMaker**: this is a market maker that uses your model trying to stay in the market. That means he is listening to spot updates, calls your model to calculate a new price for the option and then publishes the price to the market. The price should be between the bid and ask price, otherwise you give a way arbitrage and loose money. There can be more than one MarketMaker in the game.

**OptionMarket**: the market listens to the spot/quote updates and the new theoretical prices from all the market makers.

All these roles are combined in the ```TradingGame``` which is the front end you can use to test your model.


In [ ]:
from tradgame import *
import bs
import matplotlib.pyplot as plt
%matplotlib auto

The game takes a number of parameters which you can change:

| Parameter | |
|----------|---------------------------------------------|
| seed   |   Seed for the random number generator of the spot process. Keep the seed the same and you can reproduce the same spot path|
| quotewidth | Relative (in %) width of the bid/ask spread: 0.3 means that you will have the bid be 0.3% lower and the ask 0.3% higher than the mid market theoretical. Make this number smaller to make the game harder. |
| delay | The time (in seconds) the spot server waits before it publishes the next update. Make this smaller to make the game harder. Note that the speed of your computers vary so if your machine is very fast, you should probably decrease this number. |
| steps | The number of spot updates the server sends. Keep this small in the beginning so that you do not need to wait long for results. |


In [ ]:
gameParameters = {"seed" : 932748239, "quotewidth" : 0.3, "delay" : 0.001, "steps" : 100}

You give the game a list of models to run against each others. These are simply functions that need to take the following parameters:

```
func(spot, strike, tau, rate, vola)
```

Note that your model takes the number of steps as an additional parameter:

```
func(spot, strike, tau, rate, vola, steps)
```

In order to be able to run the same model with a different number of steps you can partially evaluate your function: so if you have a function ```MyBleedingFastBinomial``` you can create a function that runs your model with 100 steps by using

```
partial(MyBleedingFastBinomial, 100)
```

So your could for instance experiment with the number of steps by using the same model with different number of steps.

```
models = [ partial(MyBleedingFastBinomial, 10), partial(MyBleedingFastBinomial, 50)]
```

Note that you should not run too many models at the same time as the resources of your machine are limited. If the models start to compete for resources the results can become depended on random events.



In [ ]:
models = [bs.bscall]

It is time to start playing!

This is how we create a game and run it for the models you have setup above.

In [ ]:
game = TradingGame(gameParameters)
game.run(models)

Once the game has finished running, you can look at the results using

```
game.ranking()
```

This returns a ```DataFrame``` with the following columns:

| Column | Description |
|--|---------------|
|Name| The name of the model. If you used ```partial``` the number of steps is attached to the name. |
|Crossing| This is the number of times you either crossed the bid/ask spread or whether there was a quote update and you did not have a theoretical in the market. This is our main criterium to rank algorithms.|
|Delays| This is the average time in seconds that the model took to respond to a spot update. This can be used as a criterium if the race gets tight.|
|MidMarketness| If you are within the spread, you are probably not exactly mid-market. So this is the your average difference from mid-market. This can also me used as a criterium.|

In [ ]:
game.ranking().sort_values(by="Crossing")

You have seen the main tools that you can use to evaluate your model. If you want to see graphically how your algorithm fared, try the ```EventPlot``` below. Notice that above the plot are the names of the models together with the number of times they crossed bid/ask or missed a quote. This is updated dynamically.

In [ ]:
plt = EventPlot(game)
plt.plot()

You might want to investigate what is happening a little bit closer. In order to do that you can look at all the events in the game. You can access them by looking either at
```
game.events
```
Or in a prettier format using
```
game.getEvents()
```


In [ ]:
game.getEvents()

You should get a data frame with columns Bid, Ask, Counter, IID, Spot, Theo, Time and Type. These are:

* Bid, Ask: the quote published or NaN for model updates
* Counter: the counter of the spot/market update
* IID: -1 for spot/market updates, otherwise the ID of the model
* Spot: the spot used
* Time: the time the event occured
* Type: Quote update or Theo update

You should be able to see the updates coming from your model and connect them to a previous spot update using the Counter.

You can find the IID of each model by using
```
game.iids
```

In [ ]:
game.iids